In [31]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
import pandas as pd

In [32]:
def get_row_values(row_data):
    return [val.get_text() for val in row_data]

def fetch_treasury_yields(year):
    if year == 'ALL':
        treasury_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldAll"
    else:
        treasury_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yieldYear&year=" + "%s"% year
        
    page = requests.get(treasury_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    rates_table = soup.find(class_="t-chart")
    
    oddrow_data = get_row_values(rates_table.select(".oddrow .text_view_data"))
    evenrow_data = get_row_values(rates_table.select(".evenrow .text_view_data"))
    
    return {'oddrow': oddrow_data, 'evenrow': evenrow_data}
    

In [33]:
test_yields = fetch_treasury_yields(2021)

In [35]:
test_yields

{'oddrow': ['01/04/21',
  '0.09',
  '0.09',
  '0.09',
  '0.09',
  '0.10',
  '0.11',
  '0.16',
  '0.36',
  '0.64',
  '0.93',
  '1.46',
  '1.66',
  '01/06/21',
  '0.09',
  '0.09',
  '0.09',
  '0.09',
  '0.11',
  '0.14',
  '0.20',
  '0.43',
  '0.74',
  '1.04',
  '1.60',
  '1.81',
  '01/08/21',
  '0.08',
  '0.08',
  '0.08',
  '0.09',
  '0.10',
  '0.14',
  '0.24',
  '0.49',
  '0.81',
  '1.13',
  '1.67',
  '1.87',
  '01/12/21',
  '0.09',
  '0.08',
  '0.09',
  '0.09',
  '0.11',
  '0.14',
  '0.23',
  '0.50',
  '0.83',
  '1.15',
  '1.68',
  '1.88',
  '01/14/21',
  '0.09',
  '0.09',
  '0.09',
  '0.09',
  '0.10',
  '0.16',
  '0.23',
  '0.49',
  '0.82',
  '1.15',
  '1.69',
  '1.88',
  '01/19/21',
  '0.07',
  '0.09',
  '0.09',
  '0.11',
  '0.10',
  '0.14',
  '0.21',
  '0.45',
  '0.78',
  '1.10',
  '1.65',
  '1.84',
  '01/21/21',
  '0.07',
  '0.09',
  '0.09',
  '0.09',
  '0.10',
  '0.13',
  '0.19',
  '0.45',
  '0.79',
  '1.12',
  '1.68',
  '1.87',
  '01/25/21',
  '0.07',
  '0.08',
  '0.09',
  '0.09'

In [ ]:
def is_desc(rates_list):
    
    return (sorted(rates_list, reverse=True) == rates_list)

def avg_pct_change(rec_list):
    x = pd.Series(rec_list)
    
    return round(x.pct_change().mean(), 2)

def process_row_info(in_list):
    valMap = []
    record_list = []

    counter = 0

    for x in range(len(in_list)):

        record_list.append(in_list[x])
        counter+=1

        if (counter % 13) == 0:
            record_list[0] = datetime.datetime.strptime(record_list[0], "%m/%d/%y").strftime("%Y-%m-%d")
            record_list[1:13] = [float(i) for i in record_list[1:13]]

            record_list.extend((is_desc(record_list[1:13]), avg_pct_change(record_list[1:13])))
            valMap.append(record_list)
            record_list = []
            
    return valMap

def combined_scrapped_yields(yield_list):
    if len(yield_list) == 2:

        combined_map = []

        for key in yield_list.keys():
            combined_map.extend(process_row_info(yield_list.get(key)))
            
        return combined_map

In [ ]:
y = combined_scrapped_yields(test_yields)

In [ ]:
processed_yields = sorted(y, key=lambda x: x[0])

In [ ]:
processed_yields[0:2]

In [12]:
import mariadb

In [13]:
import sys

# Instantiate Connection
try:
   conn = mariadb.connect(
      user="root",
      password="root",
      host="localhost",
      port=3306)
except mariadb.Error as e:
   print(f"Error connecting to MariaDB Platform: {e}")
   sys.exit(1)

In [14]:
# Instantiate Cursor
cur = conn.cursor(buffered=True , dictionary=True)

In [ ]:
var_string = ', '.join('?' * len(processed_yields[0]))
var_string

In [ ]:
for records in processed_yields:
    query_string = 'INSERT INTO us_treasury_yield_rates.test_rates VALUES (%s);' % var_string
    cur.execute(query_string, records)
    conn.commit()

In [ ]:
query = f"SELECT * FROM us_treasury_yield_rates.test_rates"

cur.execute(query)

rows = cur.fetchall()

conn.close()


In [ ]:
result = []

for x in range(len(rows)):
    #print('now it is: ' + str(x))
    single_query = []

    for key in rows[x].keys():
        rec = str(rows[x].get(key))
        single_query.append(rec)

    result.append(single_query)
            
print(result[0:2])

In [ ]:
import wbgapi as wb

In [ ]:
indicators = {'NY.GDP.MKTP.CD':'GDP (current US$)'}

df = wb.data.DataFrame(indicators, 'USA', time=range(2000, 2020))

In [2]:
year_range = ""
for x in range(1990,2021):
    year_range += (str(x) + ',')
    
year_range

'1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,'

In [21]:
bea_url = "https://apps.bea.gov/api/data/?&UserID=<API KEY>&method=GetData&DataSetName=NIPA&TableName=T10105&SeriesCode=A191RC&LineDescription=Gross+domestic+product&Frequency=Q&Year=" + year_range + "&ResultFormat=json"
# data in millions

page2 = requests.get(bea_url)


In [22]:
import json
from urllib.request import urlopen
response = urlopen(bea_url)
data_json = json.loads(response.read())

In [23]:
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

raw_q_gdp = []

for x in data_json['BEAAPI']['Results']['Data']:
    
    if 'Gross domestic product' in x.values():
        y_gdp = []
        y_gdp.append(x['TimePeriod'])
        y_gdp.append(locale.atoi(x['DataValue']))
        #y_gdp.append(locale.atof(x['DataValue']))
        raw_q_gdp.append(y_gdp)

In [24]:
raw_q_gdp

[['1990Q1', 5872701],
 ['1990Q2', 5960028],
 ['1990Q3', 6015116],
 ['1990Q4', 6004733],
 ['1991Q1', 6035178],
 ['1991Q2', 6126862],
 ['1991Q3', 6205937],
 ['1991Q4', 6264540],
 ['1992Q1', 6363102],
 ['1992Q2', 6470763],
 ['1992Q3', 6566641],
 ['1992Q4', 6680803],
 ['1993Q1', 6729459],
 ['1993Q2', 6808939],
 ['1993Q3', 6882098],
 ['1993Q4', 7013738],
 ['1994Q1', 7115652],
 ['1994Q2', 7246931],
 ['1994Q3', 7331075],
 ['1994Q4', 7455288],
 ['1995Q1', 7522289],
 ['1995Q2', 7580997],
 ['1995Q3', 7683125],
 ['1995Q4', 7772586],
 ['1996Q1', 7868468],
 ['1996Q2', 8032840],
 ['1996Q3', 8131408],
 ['1996Q4', 8259771],
 ['1997Q1', 8362655],
 ['1997Q2', 8518825],
 ['1997Q3', 8662823],
 ['1997Q4', 8765907],
 ['1998Q1', 8866480],
 ['1998Q2', 8969699],
 ['1998Q3', 9121097],
 ['1998Q4', 9293991],
 ['1999Q1', 9417264],
 ['1999Q2', 9524152],
 ['1999Q3', 9681856],
 ['1999Q4', 9899378],
 ['2000Q1', 10002857],
 ['2000Q2', 10247679],
 ['2000Q3', 10319825],
 ['2000Q4', 10439025],
 ['2001Q1', 10472879],
 ['20

In [25]:
def assign_dates(qlist):
    vals = { 'Q1' : '03-30'  ,  'Q2' : '06-30'  ,  'Q3' : '09-30', 'Q4' : '12-30' }
    
    for x in range(len(qlist)):
        cal_date = qlist[x][0][0:4] + "-" + vals[qlist[x][0][-2:]]
        qlist[x].extend([cal_date])
    
    return qlist

In [26]:
assign_dates(raw_q_gdp)

[['1990Q1', 5872701, '1990-03-30'],
 ['1990Q2', 5960028, '1990-06-30'],
 ['1990Q3', 6015116, '1990-09-30'],
 ['1990Q4', 6004733, '1990-12-30'],
 ['1991Q1', 6035178, '1991-03-30'],
 ['1991Q2', 6126862, '1991-06-30'],
 ['1991Q3', 6205937, '1991-09-30'],
 ['1991Q4', 6264540, '1991-12-30'],
 ['1992Q1', 6363102, '1992-03-30'],
 ['1992Q2', 6470763, '1992-06-30'],
 ['1992Q3', 6566641, '1992-09-30'],
 ['1992Q4', 6680803, '1992-12-30'],
 ['1993Q1', 6729459, '1993-03-30'],
 ['1993Q2', 6808939, '1993-06-30'],
 ['1993Q3', 6882098, '1993-09-30'],
 ['1993Q4', 7013738, '1993-12-30'],
 ['1994Q1', 7115652, '1994-03-30'],
 ['1994Q2', 7246931, '1994-06-30'],
 ['1994Q3', 7331075, '1994-09-30'],
 ['1994Q4', 7455288, '1994-12-30'],
 ['1995Q1', 7522289, '1995-03-30'],
 ['1995Q2', 7580997, '1995-06-30'],
 ['1995Q3', 7683125, '1995-09-30'],
 ['1995Q4', 7772586, '1995-12-30'],
 ['1996Q1', 7868468, '1996-03-30'],
 ['1996Q2', 8032840, '1996-06-30'],
 ['1996Q3', 8131408, '1996-09-30'],
 ['1996Q4', 8259771, '1996-1

In [27]:
def mil_to_tril(qlist):
    
    for x in range(len(qlist)):
        qlist[x].extend([round(qlist[x][1] / (10**6),2)])
    
    return qlist

In [28]:
mil_to_tril(raw_q_gdp)[0]

['1990Q1', 5872701, '1990-03-30', 5.87]

In [29]:
var_string2 = ', '.join('?' * len(raw_q_gdp[0]))
var_string2

'?, ?, ?, ?'

In [30]:
for records in raw_q_gdp:
    query_string = 'INSERT INTO us_treasury_yield_rates.test_gdp VALUES (%s);' % var_string2
    cur.execute(query_string, records)
    conn.commit()